# NERHYMUS Preprocessing
## Step-by-step guide
>This notebook will go through the steps of EEG preprocessing as done for the NERHYMUS study by Katerina Kandylaki.
You mostly will not have to actually do much but run the sections of code one by one (never at once, unless you want your computer to crash). However, pay attention to steps 2, 3, 4, 6 and 10, in which you will have to alter the code or save the outputs to your participant folder.

### Steps
><a href="#1.-Import-modules">1. Import modules</a><br><a href="#2.-Settings">2. Settings<a/><br><a href="#3.-Import-raw-data">3. Import raw data<a/><br><a href="#4.-Bads">4. Bads<a/><br>
<a href="#5.-Resampling">5. Resampling</a><br><a href="#6.-Filtering">6. Filtering</a><br><a href="#7.-Cropping">7. Cropping</a><br><a href="#8.-Interpolation">8. Interpolation</a><br><a href="#9.-Re-referencing">9. Re-referencing</a><br><a href="#10.-ICA">10. ICA & artefacts</a><br>


## 1. Import modules

[<a href="#Step-by-step-guide">back to top</a>]

>Before processing our data, we need to make sure we have the tools to properly work with the datasets. Each of the following libraries adds a function/ability to the basic functions included in Pythons. We don't need to understand each function in detail but make sure that you have each of the following installed (either via Anaconda Prompt or via the Anaconda Navigator). The "as" command simply renames the imported library for easier typing later-on in the code. The "from...import" command specifies that we want to import only a specific module within a library.

#### NumPy, Pandas, OS, ast library
These four libraries deal with the structuring of data within python (lists, matrices, tables, timeseries, trees).
#### MNE 
This module is the core of our EEG preprocessing (although it is also used for other similar neuroimaging techniques (MEG, NIRS, ...)). It helps us e.g. vizualize EEG data and do the Independet Component Analysis which is essential later in determining which EEG activity stems from eye movements or other artefacts.
#### Pickle 
"Pickling" in python refers to the creation of new files based on output from running a piece of code, so that it can be used again later-on without having to create the same output again. This helps provide consistency in our processing. This process is also refered to as  serializing and de-serializing a Python object structure.
#### Comma Separated Values (CSV)
CSV format is commonly used to  import and export data from spreadsheets and databases. Here, for instance, it is used to read txt files. It can also help produce code/output compatible with excel.
#### Logging 
Logging modules helps prevent the console from being spammed with debug messages.

In [1]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import join
import mne
import ast
import pickle
import csv

In [2]:
import logging
logging.getLogger().setLevel(logging.WARNING);

## 2. Settings

[<a href="#Step-by-step-guide">back to top</a>]

### 2.1 Programmer & path

>Every person using this preprocessing code will have their participant data stored in different exact locations and we need to tell python this by defining the location of our EEG data files. To do so, add yourself to list of programmers by copying one of the previous IF-statements and inserting your name and the file path where you store your participant data. You can retrieve the full path to the EEG folder by clicking the white space next to the path description in your explorer (marked in green) and simply copying the marked path.

<img src="images\0folder.png" style="width:872px;height:345px"/>

#### Add yourself to the list of programmers & the path to the EEG folder on your device

In [ ]:
programmer = 'Kobus'
if programmer == 'Kobus':
    the_folder = 'C:\\internship2\\'
if programmer == 'Katerina':
    the_folder = 'C:\\Users\\katerina.kandylaki\\Documents\\NERHYMUS\\Experiment\\Data\\EEG'

In [ ]:
programmer = 'Lea'
if programmer == 'Kobus':
    the_folder = 'C:\\internship2\\'
if programmer == 'Katerina':
    the_folder = 'C:\\Users\\katerina.kandylaki\\Documents\\NERHYMUS\\Experiment\\Data\\EEG'
if programmer == 'Lea':
    the_folder = 'C:\Users\leano\OneDrive\Documents\Master RM CCN NP\Year 1\Research Elective NERHYMUS\EEG'

#### Possible Error

In [ ]:
SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape

Try changing changing backslashes to double backlashes in path or entering path in r' ' :

In [ ]:
if programmer == 'Lea':
    the_folder = 'C:\\Users\\leano\\OneDrive\\Documents\\Master RM CCN NP\\Year 1\\Research Elective NERHYMUS\\EEG'

### 2.2 Participant Information

>For python to know which dataset to work on, we have to specify the condition (EVC, SMA or sham) and participant number (e.g. P32, M08). With that knowledge it can move from the general EEG folder (the_folder) into the correct condition and to the correct subject folder. Using the join function we enter this information to a new path named fullpath, which can be refered to for the rest of the processing.
The hdrfile variable created in the last line refers to one of the data files and when we later read this .vhdr file, the same command will include commands that also read both an .eeg and a .vmrk file. For this to work properly, all three files must be in the same folder, namely the "fullpath".

#### Enter the correct condition and participant number for what you want to work on

In [ ]:
condition = "EVC"
subject = "M19" 

fullpath = join(the_folder,condition,subject)

hdrfile = [f for f in listdir(fullpath) if f.endswith('.vhdr')][0]

### 2.3 Filtering

>Here we specify how the EEG data will be filtered later-on in Step 6. We want to remove very high and very low frequencies as they are unlikely to be caused by meaningful brain activation. We apply a lowpass value of 30Hz (lp = 30) and a highpass value of 1Hz (hp = 1). You shouldn't have to change these values in your code.
<br> <br>
A high-pass filter of 0.1-0.5Hz is recommended in order to minimize slow drifts affecting the final data (these can e.g. be caused by sweating). In our case we need to keep the high-pass filter at 1Hz (explained below) but since we look at quite a broad frequency range, it is unlikely that we will miss important brain activity. If we were interested in e.g. delta waves, this would be more problematic, since those occur between 0.5Hz and 4Hz. In that case a change from 0.1 to 1hz would make a much bigger difference.
<br> <br>
The low-pass filter is applied to remove high-frequency activity that is most likely caused by noise and muscle artefacts and occurs at 30Hz and above.
<br> <br>
The lb and hb values underneath each bandpass filter refer to the transitional bandwidths. These determine the "slope" with which frequencies are filtered out. In short, it can be problematic to simply cut off the data at a certain frequency. You should rather have some form of transition at the cut-off point. How steep this cut-off from the pass filter should be, depends on the frequency. The higher end (low-pass frequency = 30Hz) should be cut off using a bandwidth of a fourth of the frequency (lb = 7.5Hz). On the lower end it is a bit more tricky for us (high-pass frequency = 1Hz). It is recommended to have a low-end bandwidth of 10% of that but for us the bandwidth cannot be lower than 1Hz, which is why both the filter & the bandwidth are kept at 1Hz.
<br> <br>
Note: there are alternative versions of the code that specify the frequencies of interest more specifically; for instance only look at frequencies between 0.1-8Hz.

In [ ]:
#INSERT LOWPASS VALUE
lp = 30
lb = lp/4

#INSERT HIGHPASS VALUE #1/10th of hp but never lower than 1!
hp = 1
hb = 1

### 2.4 Sampling rate
>The sampling rate describes to the number of "images" or data acquisitions from all electrodes per second and determines the temporal resolution of one's EEG analysis. The higher the sampling rate the better the signal-to-noise ratio will be, which is the ultimate goal of the preprocessing. Generally, a rate between 500Hz and 2000Hz are sufficient for all analyses. One big disadvantage of such a high sampling rate is the added space used per datafile, which slows down computation and data transfer processes. A solution to this problem is to downsample the datasets to a more managable size after completing the acquisition with a higher sampling rate.
<br><br>
In our case, during the acquisiton, a sampling rate of 500Hz was used, which is the general minimum recommended. Keep in mind, we are using ~45 minutes of continuous EEG data, so a higher sampling rate would significantly increase the amount of data we need to store and transfer. From the 500Hz, we then downsample to 125Hz, which, after trying it with 250Hz as well, was found to be the most optimal to provide sufficiently good data, while not compromising computational time.
<br><br>
Lastly, we create a variable "ICA_NAME_SPECS" which combines information on the previously chosen values for the Independent Component Analysis later in Step 9. This variable resembles the name of a data file, which we will create during our processing and will be able to find in our data folder after processing: "_Select_0.1-30Hz_R125_with_interpolation-ica.fif",

In [ ]:
#INSERT RESAMPLE FREQUENCY
sample_freq = 125
sfreq=sample_freq
#ICA SPECS
ICA_NAME_SPECS = '_SELECT_1-30Hz_R'+str(sample_freq)+'_with_interpolation-ica.fif'

## 3. Import raw data

[<a href="#Step-by-step-guide">back to top</a>]

### 3.1 Easycap
>With this section we specify the details on the cap placement and electrodes for MNE. This first figure simply shows the template that most closely fits our electrode placement.

In [ ]:
mon = mne.channels.read_montage(kind='easycap-M1', ch_names=None, path=None, unit='m', transform=False)
mne.viz.plot_montage(mon)

<img src="images\1_easycap.png" style="width:350px;height:400px"/>

### 3.2 Raw plot
>In this section we define the raw data and our channel labels. We ask MNE to read the .vhdr file from before, which is a so-called "text header file" containing "meta data". With the "preload = true" command, we tell MNE to preload data into memory, which quickens data manipulation but uses more memory.
Within an EEG template you can add additional non-EEG channels under "misc" for miscellaneous. In our case, we add a channel, named "stim", that carries the audio stimulus. We then add the information that the electrodes 64 & 65 are our eye electrodes (if you help out during the EEG setup, you will recognize these).
<br><br>
"picks" tells MNE that we are dealing with EEG data (EEG=True), and "sound_ch" and "eog" are then connected to the misc channels and EOG channels, respectively.
<br><br>
Once we specified all of our channels, we can plot the raw data using two commands, creating both a concise graph to save in the participant folder, as well as a bigger detailed look into the raw data.



In [ ]:
raw = mne.io.read_raw_brainvision(join(fullpath, hdrfile), preload=True, misc=('Stim',), eog=('64','65'))
picks = mne.pick_types(raw.info, meg=False, eeg=True, misc=False, stim=False)
sound_ch = mne.pick_types(raw.info, meg=False, eeg=False, stim=False, misc=True)
eog = mne.pick_types(raw.info, meg=False, eeg=False, eog=True, misc=False, stim=False)

#Plot to check the messy data
raw.plot()

raw.plot_psd()

#### Save the concise graph in the participant folder

<img src="images\2rawbig.png" style="width:928px;height:378px"/> 

<img src="images\3rawsmall.png" style="width:473px;height:439px"/>

## 4. Bads

[<a href="#Step-by-step-guide">back to top</a>]

### 4.1 Preparation

>Prior to this preprocessing, you will have received the data folders and in each folder there should be an eegbads_lab.txt file and maybe even already a bads.txt file. These include information on which channels will be excluded from the analysis. If there is no .txt file named "bads" you need to create one.
In that you have to type the rejected electrode names (as they are written in the eegbads_lab file!) into quotation marks ('') and into square brackets ([]). When there are multiple electrodes you separate them with a comma (,). I added an example of what the final bads.txt file could look like below. Make sure that, when saving the txt file you only call it "bads" and not "bads.txt". The .txt will be added to the file path automatically.

#### Create the bads.txt file

<img src="images\badstxt.png" style="width:600px;height:250px"/>

##### What if I get an error:
Check bads.txt file for possible typing errors, e.g. missing a ' or , or ]    
Here, also make sure to type out the bads correctly, as sometimes letters should be in upper or lowercase (e.g. 'T8', 'Fp2', 'AF7', 'POz'). Simply use the same spelling as in eeglab_bads.txt file.


### 4.2 Reading the bads
>When we have the bads defined and in their txt file, we can read this file into python by running this section. Manually running raw.info['bads'] should then show you the same content as is in the bads.txt file.
<br><br>
You can check if ALL the correct channels were taken out in the output of Step 6 (deleted channels will appear in a fainter grey).

In [ ]:
bads_file = open(join(fullpath, 'bads.txt'),'r')
bads = bads_file.readline()
bads_file.close()
bads = ast.literal_eval(bads)
raw.info['bads']=bads

### 4.3 Checking important channels
>Now we want to check if any of our essential electrodes necessary for the analysis are rejected. For that we define variables that provide information on whether an electrode is bad (xxx_bad). They are set to 0 because we initially assume that they are not rejected but each will be checked for that condition in the next step.
In the for-loop, the entries in the bads.txt file are gone through one by one and if one of the rejected electrodes corresponds to either one of the frontoparietal channels (Fp1/Fp2), the EOG channels (63/64) or the reference channels (TP9/TP10), for that channel the xxx_bad variable will be marked as true ( = 1). Fp1 and Fp2 in particular are important for later checking of the eyeblink artefacts.

In [ ]:
Fp1_bad = 0
Fp2_bad = 0
eog_bad = 0
TP1_bad = 0
TP2_bad = 0
for x in range(0,len(raw.info['bads'])):
    if raw.info['bads'][x] == 'Fp1':
        Fp1_bad = 1
    if raw.info['bads'][x] == 'Fp2':
        Fp2_bad = 1
    if raw.info['bads'][x] == '63':
        eog_bad = 1
    if raw.info['bads'][x] == '64':
        eog_bad = 1 
    if raw.info['bads'][x] == 'TP9':
        TP1_bad = 1 
    if raw.info['bads'][x] == 'TP10':
        TP2_bad = 1 

### 4.4 Do we have at least one good reference?
>Lastly, the reference channels TP9 and TP10 (also called offline channels) are checked for whether one or both of them are rejected. If one of them is rejected, only the other is kept as the reference. If both are rejected, the dataset is useless.
Reference electrodes are necessary so we have something to measure baseline activity, which we can compare all the other activity changes too. Typically electrodes on the mastoids (bone behind the ear) are used to place the reference electrodes (again, if you're helping in EEG setup, you'll recognize this.)

In [ ]:
if TP1_bad == 0 and TP2_bad == 0:
    REFs = ['TP9','TP10']
if TP1_bad == 0 and TP2_bad == 1:
    REFs = ['TP9']
if TP1_bad == 1 and TP2_bad == 0:
    REFs = ['TP10']
if TP1_bad == 1 and TP2_bad == 1:
    REFs = [None]
    print('NO GOOD REFERENCE! BOTH REFERENCES ARE BAD! USELESS DATASET! *In doubt check TP9 and TP10 quality*')

## 5. Resampling

[<a href="#Step-by-step-guide">back to top</a>]

>Here we complete the re-sampling as specified in step 2.4 in order to reduce the size of the dataset.

In [ ]:
raw.resample(sfreq=sample_freq)

## 6. Filtering

[<a href="#Step-by-step-guide">back to top</a>]

>Using the high and low pass-values defined in Step 2.3, we finally filter our data and create new plots showing the filtered data. You should see a clear difference to the previous plots that contained the raw data. You can also see the role of the bandwidth, in that the drop after 30Hz is there but somewhat "slowed down" by the curve. Additionally, in the first, larger graph you can check whether all the bad channels were removed correctly (a more faint grey line will appear for the rejected channels, pointed out here with the red circles).

In [ ]:
print("Filtering data between 1 and 30 Hz")
low_pass = lp #y
high_pass = hp #x
raw = raw.filter(high_pass, low_pass, picks=picks, n_jobs=-1, fir_design='firwin')


raw.plot()

raw.plot_psd()

#### Save the concise graph in the participant folder

<img src="images\4filteredbig.png" style="width:926px;height:378px"/>

<img src="images\5filteredsmall.png" style="width:481px;height:445px"/>

## 7. Cropping

[<a href="#Step-by-step-guide">back to top</a>]

>Here we cut the data down to the parts where participants listen to poems/stories by removing the sections of testing where they answer questions or have a break. We only want to feed the listening-data into the ICA, in order to make it more specific. If we included the whole dataset, there would be a lot of redundant components related to artefacts because of the breaks (moving, stretching, having a sip of water) and from reading/answering the questions, which reduces the ability of the ICA to detect more subtle artefacts (e.g. eye blinks, muscle artefacts).

### 7.1 Special conditions

>During some testing sessions, things can fail or go wrong so you have to restart part of a session. This leads to there being more data than there should be, which we need to cut out.
This is the case for participants P02 and M16. For P02, for instance, in the SMA condition, the experiment had to be restarted, leading to more measurements than there should be for this dataset. As you can see, we exclude some of their annotations, only leaving the correct amount of data for the analysis.
<br><br>
So, we are not excluding the whole testing session, but only the first two annotations. Also, the if statement makes sure that this only happens for P02 and M16 in those conditions where there was an issue during the acquisition. These lines should always remain uncommented.  

In [ ]:
if condition == "SMA" and subject == "P02":
    raw.annotations.delete(1)
    raw.annotations.delete(1)

if condition == "EVC" and subject == "M16":
    raw.annotations.delete(46)

### 7.2 Annotations
>Annotations (also called markers/trigger values) mark the timepoints within the data at which certain stimulus or response events occur. Usually they are arbitrary numbers with no other inherent meaning other than being a label/name for an event. They help us divide the EEG data into sections and cut it down to only the desired ones.
<br><br>
1 & 2 = response
<br><br>
3 = question shown on screen
<br><br>
4 = start of audiofile
<br><br>
8 = end of audiofile
<br><br><br>
For us markers "4" and "8" are important because they represent the sections of an audio file, which is exactly the way we want to cut the data. You can check the order of the markers across the dataset by running "raw.annotations.description" in the console, if there is a suspicion that something is wrong in the number or order of events/annotations.

In [ ]:
ant = raw.annotations 
timestamps = ant.onset
response = ant.description
responseval = [0]*len(response)
for xc in range(0,len(response)):
    if '4' in response[xc]: 
        responseval[xc] = 4
    elif '8' in response[xc]:
        responseval[xc] = 8
    else:
        responseval[xc] = 0

startcounter = 0
for x in range(0,len(responseval)):
    if responseval[x] == 4:
        startcounter +=1
blocks = [0]*startcounter

errorprooftime = [0]*(len(blocks)*2)
xyy = 0
for xy in range (0, len(responseval)):
    if responseval[xy] == 4:
        errorprooftime[xyy] = xy
        xyy +=1
    elif responseval[xy] == 8:
        errorprooftime[xyy] = xy
        xyy +=1
   

<img src="images\raw.annotations.description.png" style="width:645px;height:434px"/>

### 7.3 Finding the right audiofiles (background)

In the following sections, python will go through various documents in the "details" folder that exists within the EEG folder. In this folder there is, essentially, information on which audiofiles were used during which sessions taking into account conditions and participants. You do not need to know what exactly python does here to run it, but I will try and give some insight into the data and files it uses because this information can be valuable when working on troubleshooting errors.

>The lines of code assigning the “Durationblock” variable are initialising this variable, and assigning the values of the length of each block according to “timestamps”.
<br><br>
The next block of code reads in the theobeat feature from the pickle file ending in RZd.pickle, which is found under the details directory. More information about pickling can be found in the <a href="#-Sources-&-additional-materials">Sources & additional materials</a>.

In [ ]:
Durationblock=[None]*len(blocks)
y2 = 0
for y in range(0,len(blocks)):
    fetch1 = errorprooftime[y2]
    fetch2 = errorprooftime[y2+1]
    Durationblock[y] = timestamps[fetch2] - timestamps[fetch1]
    y2 +=2    

insertedFTTlabel = 'theobeat'

pickle_in = open(join(the_folder +'/details/'+str(sfreq)+'', insertedFTTlabel +"_RZd.pickle"),"rb")
insertedFTT = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
pickle_in = open(join(the_folder +'/details/',"labelsandstampsR.pickle"),"rb")
labelsandstampsR = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open(join(the_folder +'/details/',"OrderOfAudio.pickle"),"rb")
OOA = pickle.load(pickle_in)
OrderOfAudio = OOA
pickle_in.close()

<img src="images\6pickles.png" style="width:480px;height:330px"/>

>This part reads the Lists_for_EEG.xls file in the details folder. In it, we find information on which list of audiofiles is used per combination of participant by condition. Here, for example, the participant was exposed to lists C4, A2 and B3 across their 3 EEG sessions. We will see in a moment what exactly that means.

In [ ]:
countnumbercondition = 0

file_name_lists = the_folder + '/details/Lists_for_EEG.xls' #has the list codes per session.
xl_file1 = pd.ExcelFile(file_name_lists)
dfs_lists = pd.read_excel(file_name_lists, sheet_name=None)
listchecker = dfs_lists['Sheet1']
participant_session_listinfo =   np.array([[None,None,None,None]]*100)
for x in range(0, len(listchecker)):
    for y in range(0,4):
        participant_session_listinfo[x][y] = listchecker.iat[x,y]

<img src="images\7lists_for_EEG.jpg" style="width:365px;height:320px"/>

>So now we know which potential audiofiles our participant was exposed to, but we still need to specify which of the 3 possible lists was presented during the particular session we are processing. In other words, python needs to know which TMS condition (SMA, EVC or sham) we are working on. For that, python now reads the TMS_condition.xls file, also within the details folder, and finds which TMS condition applies to the current analysis. In this case, the EVC condition for M19, which happens to be the 2nd session.

In [ ]:
file_name_TMS = the_folder +'/details/TMS_condition.xls' #has the TMS conditions per session
xl_file2 = pd.ExcelFile(file_name_lists)
dfs_lists2 = pd.read_excel(file_name_TMS, sheet_name=None)
listchecker2 = dfs_lists2['Sheet1']
participant_session_tmsinfo =   np.array([[None,None,None,None]]*100)
for x in range(0, len(listchecker2)):
    for y in range(0,4):
        participant_session_tmsinfo[x][y] = listchecker2.iat[x,y]

conditioncode = condition
for xas1 in range (0,len(listchecker2)):
    if participant_session_tmsinfo[xas1][0] == subject:
        for xas2 in range (0,4):
            if participant_session_tmsinfo[xas1][xas2] == conditioncode:
                conditionnumber = countnumbercondition
            if participant_session_tmsinfo[xas1][xas2] != conditioncode:
                countnumbercondition +=1


<img src="images\8TMS_condition.jpg" style="width:405px;height:277px"/>

>This following part loops through the list of list versions (eg. B1, C5 etc.) to see which list was used in this session number. In our example, from the excel files we can already see that we are processing the 2nd EEG session for M19, during which they were exposed to the audiofiles from list A2. Once we know this, python stores this information in the variable "listdetails". After running this section you can manually run "print(listdetail)" or "print(participantlist)" and it should give you the name of the list (Here: A2).
<br><br>
Note: "listversion" tells python which version of the lists is used (Here: A) and "listnumber" represents the specific list within all the A-lists (basically, which column in the following excel file) that contains the specific audiofiles relevant for us (Here: 2).

In [ ]:
for xas3 in range (0,len(listchecker)):
    if participant_session_listinfo[xas3][0] == subject:
        participantlist = participant_session_listinfo[xas3][conditionnumber]
            
listdetail = participantlist
listnumber = int(listdetail[1])
listversion = listdetail[0]  

<img src="images\10_print(participantlist).png" style="width:378px;height:148px"/>

>So now we know (and python knows) which audiofiles the participant listened to in the session we are processing, python will open the correct version of lists (Here: A) and read the column that applies for us (Here: 2nd). With that, we finally know the audiofiles our participant listened to. The labels of these files are stored as a list, which will be returned by python. Otherwise, you can check it manually using the "print(labels)" command. 
<br><br>
Note: In the excel file it might seem confusing that we refer to the 2nd list but the B-column but remember, you don't actually have to look at these files when running the code (unless you need to dig deeper for some troubleshooting) so do not worry about this too much.

In [ ]:
with open(the_folder +'/details/Lists_' + listversion + '_pseudorandomised.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=';')
    labels = []
    colors = []
    for row in readCSV:
        label = row[listnumber-1]
        labels.append(label)
        
    print(labels)
    

FTTbasedstartandend = [0]*(2*len(labels))
yc=0
for x in range (0, len(labels)):
    
    FTTbasedstartandend [yc] = timestamps[errorprooftime[yc]]
    FTTbasedstartandend [yc+1] = timestamps[errorprooftime[yc]] + labelsandstampsR[labels[x]]
    yc +=2

getnumber = [0]*len(Durationblock)
for x in range(0, len(Durationblock)):
    for y in range(0, 40):
        if labels[x] == OrderOfAudio[y]:
            getnumber[x] = y 

<img src="images\9Lists_A_pseudorandomized.png" style="width:532px;height:240px"/>

<img src="images\11_print(labels).png" style="width:523px;height:65px"/>

### 7.4 Cutting the EEG signal
>To actually cut the EEG signal, we use the length of the audio as a marker. We take the timestamp of annotation 4 (start of audio) and then we cut the EEG signal after that depending on how many frames of our audio feature we have. This ensures that all participants and sessions are cut (or concatenated) in the same manner, as sometimes there are slight differences in the number of frames (e.g. 2-5 more frames for eeg than needed). This would be a problem for further analysis (i.e. computation of the Temporal Response Function) because both signals (EEG and audio feature) need to have the same sampling frequency and the same length. FFT5 is the theobeat (theoretical beat), which represents the "rhythm" of a story or poem and can be used to identify frames or sections of the audiofiles. This information is used to correct the length of the EEG data equally for each participant according to the audiofiles they are listening to.

In [ ]:
insertedFTT_len = []
FTT5_ws_poems = []
FTT5_ws_stories = []          
fillcheck_stories = 0
fillcheck_poems = 0
xcount1 = 0            
rawpartcount = 0
for x in range(0, len(Durationblock)):
    raw_part = raw.copy().crop(FTTbasedstartandend[xcount1], (FTTbasedstartandend[xcount1] + ((len(insertedFTT[getnumber[x]])/sfreq))))

    if rawpartcount == 0:
        insertedFTT_len = list(insertedFTT[getnumber[x]])
        raw_extendhere = raw_part
        rawpartcount += 1
        difflen = len(insertedFTT_len) - len(raw_extendhere)
        while len(raw_extendhere) > len(insertedFTT_len):
            insertedFTT_len.append(sum(insertedFTT[getnumber[x]])/len(insertedFTT[getnumber[x]]))
    if rawpartcount == 1:
        insertedFTT_len.extend(list(insertedFTT[getnumber[x]]))
        raw_extendhere.append([raw_part])
        difflen = len(insertedFTT_len) - len(raw_extendhere)
        while len(raw_extendhere) > len(insertedFTT_len):
            insertedFTT_len.append(sum(insertedFTT[getnumber[x]])/len(insertedFTT[getnumber[x]]))
    xcount1 +=2    
    
    
raw = raw_extendhere   

### 7.5 Help! Errors!

##### Error with reading the excel files
>One potential error can occur regarding your version of xlrd, which is a python library that reads excel files into python. Newer versions (2.0.0 or newer) of this library can only read .xls files, and not .xlsx files. You can check which version you have installed in the Anaconda Navigator. In this case, you may need to change path details for the Lists_for_EEG and TMS_condition paths in this section of the code (for me those are in line 231 and 241). Simply change the path ending from .xlsx to .xls. If this still doesn't work, you might have to check the details folder for whether the 2 excel files there are already .xls files. Otherwise, open them and re-save them as .xls files.


##### Error in cutting the data
>Follow the previous steps to manually find the correct list of audio files for your participant and make sure it matches the output given by python. Here are some commands to check that python has correct information on the audiofiles:
<br><br><br>
print(labels) = should give the names of the audiofiles
<br><br>
len(labels) = should be equal to the number of audiofiles
<br><br>
len(FTTbasedstartandend) = should be double the length of labels (counts start + end for each audiofile)
<br><br>
len(timestamps) = shows you how many events in total were detected


## 8. Interpolation

[<a href="#Step-by-step-guide">back to top</a>]

>Earlier we identified the bad channels for this participant dataset. There are multiple ways you can deal with bad channels. You can simply drop them or you can interpolate them, which means that values for the bad electrodes are calculated based on the neighbouring electrodes. Generally, the ICA can only find as many independent components as we have channels, so if we only removed bad channels, the analysis would be different across participants and testing sessions. So by interpolating we have information for all electrodes which ultimately increases the precision of the ICA in in Step 10.

### 8.1 Writing bads into txt file again

In [ ]:
thefilez= open(join(fullpath, 'bads.txt'),'w')
thefilez.write(str(raw.info['bads']))
thefilez.close()

### 8.2 Interpolation
>As you can see the part on dropping bad channels is commented out, because (in this version of the code) we do not simply drop the bad electrodes but interpolate them. Underneath we find the command that actually runs the interpolation.

In [ ]:
####drops the bad channels (only do this for preprocessing ICA!)
#drop_bads = raw.info['bads']
#raw = raw.drop_channels(ch_names=drop_bads)

#### Interpolating bad channels
print("Interpolating bad channels")
raw.interpolate_bads()


## 9. Re-referencing

[<a href="#Step-by-step-guide">back to top</a>]

>During the acquisition we used the FCz electrode as a reference (online reference) but for the analysis/processing we need to change this because using the FCz is suboptimal. As we already set before, we chose the TP9 (left mastoid) and TP10 (right mastoid) electrodes behind the ear as our offline references. We now have 2 offline references but python will use their average as the final reference.

In [ ]:
print("Re-referencing")

raw, ref = mne.io.set_eeg_reference(raw, ref_channels=REFs)

## 10. ICA

[<a href="#Step-by-step-guide">back to top</a>]

>Note: If you have already run an ICA for a participant DO NOT run it again! Simply load the saved file of the first ICA via the load command below.
##### Also remember to save all of the output in the participant folder

### 10.1 run, save, load the ICA
>The ICA decomposes the signal into temporally and spatially consistent patterns of signal, which we call components. It doesn’t know how many components to try to find, so you need to specify that. Katerina has decided that a number of 40 components does the job. These couple of lines of this section are the core of the ICA. The first line specifies the ICA or finds the components and thus kind of creates the tools for he analysis. The second line then fits or applies that specification to our data.

>Note: If you would like to understand what exactly a decomposition means, you can find read more about Fourier Transforms. These are our <a href=https://mne.tools/stable/generated/mne.preprocessing.ICA.html>specifications for the function we use in MNE Python</a>.

In [ ]:
ica = mne.preprocessing.ICA(n_pca_components=50, max_pca_components=40, n_components=40, random_state=23)

ica.fit(raw)

>After running the ICA, the results are saved in a .fif file (which we already named in Step 2.4 but will now appear in the participant folder).

In [ ]:
ica.save(join(fullpath, subject + ICA_NAME_SPECS))

>You might want to re-run the ICA, e.g. to produce the graphs of more components than you did previously. However, you cannot simply run the whole ICA multiple times, as each ICA decomposition is an estimation and thus it is unique every time. This means when you run an ICA decomposition and decide to exclude components, you need to be able to load the exact same ICA decomposition when you discard components.
<br>
For that you can use the "mne.preprocessing.read_ica" command. This allows you to load a saved ICA from a previous run of the analysis. If you closed and reopened Spyder, make sure to run sections 1 and 2 again (and check that you picked the right participant). Then run this section to load the previous ICA (make sure to remove the # before it) and then you can work with the plotting commands below.

In [1]:
ica = mne.preprocessing.read_ica(join(fullpath, subject + ICA_NAME_SPECS))

### 10.2 Plotting the ICA

>After you ran the ICA, you can plot components individually or immediately create multiple plots at once.
<br> The interpretation of these plots is explained below.
>##### Components
This displays all individual components' topographies together across two figures.
>##### Properties
This is used to show more information about a single component.
>##### Overlay
This creates an image of data when ICA is substracted and with original raw to see the difference.
>##### Sources
Plots the ICA components over time.

In [ ]:
ica.plot_components()
ica.plot_properties(raw, picks=12)
ica.plot_overlay(raw)
ica.plot_sources(raw)

>##### Properties for selectpicks
With this section you can get the properties of multiple components in one step. Just fill in which components you would like to look at in more detail in the selectpicks = [] and run this section.

In [ ]:
selectpicks = [0,1,5,29,38]
for x in range(0,len(selectpicks)):
    ica.plot_properties(raw, selectpicks[x])

>The goal of the ICA is to identify artefacts to be discarded. You have the choice between manually deciding which components are artefacts (step 10.2) and should be removed based on the output from the ICA or you can let python decide automatically using step (step 10.3). Ideally, you should decide manually and then compare your outcomes with what python identifies as artefacts. Either way, this section saves your chosen artefact components in a file called "bad_ics.txt". As you can see in the second line it enters a list into this txt file. This list needs to be created by you after deciding which components should be discarded as artefacts.
<br><br>
Note: As the code is written the lists is called "eog_comps" but you could create a new list with a different name and change it in the .writelines command line.

In [ ]:
eog_comps = [x,x,x,x,xx,xx,xx,...]

In [ ]:
bad_ics_file = open(join(fullpath, 'bad_ics.txt'),'w')
bad_ics_file.writelines(["%s\n" % item for item in list(eog_comps)])
bad_ics_file.close()

## 10.2 Interpreting the output of the ICA
>The goal of the ICA is to identify eye movements and other artefacts. To know which of the components from the ICA represents artefacts and not brain activation, we need to know how to interpret the output of the ICA.

### 10.2a Summary of all components
>The first images are a summary of all the components detected by the ICA. It's a good first overview to get an initial impression of the components and to maybe notice similar/mirrored/related components. The first few components usually include eye movement artefacts, which can typically be identified quite clearly. The topographical maps also show clearly when a component relates to noise from a single channel.

<img src="images\14_all_components.png" style="width:838px;height:334px"/>

### 10.2b Properties of each component
>Together these properties tell us if a certain independent component is an artefact and informs our choice about whether or not to take it out (clear topography abour eyes/single electrode + lots of data affected + consistent occurence)
<br><br>
Note: the output given here was done with another version of the code that filters the data between 0.1 and 8Hz and using different participant data.
>#### Topography maps for ICA component
These maps shows among which channels the ICA detected common activation. Usually the first few ICs are eye/frontal-related components. Alternatively, it can be interpreted as showing which channels will be affected if the component is excluded.
>#### Epochs image
In an ERP experiment, this graph would show activity related to a certain event but epochs are not really meaningful for us. We must interpret it as continuous data (each line continuing into next). The red and blue marks show us how often and when in the experiment this particular component occurs. Again, these parts of the data would be affected if the component was to be removed.
>#### Epochs variance
In short, this graph shows whenever this component occurred, how different it is from the previous one. The y-axis gives us the signal strength for this component and the x-axis the epoch (which again is not important for us, so for us it simply shows all the instances of the component over time). Eyeblinks are usually quite consistent with	aligned/overlapping dots/dots that make a line. Usually they also appear at the same strength every time. If there is more variety and if it is unclear whether the component comes from the frontal electrodes, you should rather keep it in.
>#### Spectrum
Lastly, the spectrum analysis tells us which frequencies are impacted.

<img src="images\15_ICA_details.png" style="width:936px;height:307px"/>

##### Other sources
I only provided short explanations to give an initial impression of the outputs. Other authors have written more detailed explanations on how to identify different types of artefacts after an Independent Component Analysis. This website even has a way to practice identification of components, which I highly recommend (even if the output looks somewhat different than ours): https://labeling.ucsd.edu/tutorial/labels


## 10.3 "synthesized" EOG components

>To identify which independent components belong to eyeblink artefacts, we can manually pick out our choices (as described before). However, we can also check our choices by comparing them to artefacts chosen by the programme itself. This is what happens in this part of the code. We use 3 different reference channels to identify eye-related artefacts (given that they haven't been removed as bad channels in the analysis previously (Note: if xxx_bad == 0). This method is also called "synthesized" EOG and identifies which ICs correlate with EOG-related channels (FP1, FP2, EOG), which we can also call seeds in the context of ICA.

>At first, we define an empty list, in which the chosen/detected bad components will be entered.

In [ ]:
eog_comps = []

>Next the ICs are correlated with FP1. A new txt file is created displaying them. In python it is called "thefile" and in your data folder it will be called "eog_comps_basedonFp1.txt".

In [ ]:
if Fp1_bad == 0:
    eog_comps1 = ica.find_bads_eog(raw, ch_name='Fp1')
    thefile= open(join(fullpath, 'eog_comps_basedonFp1.txt'),'w')
    thefile.writelines(["%s\n" % item for item in list(eog_comps1[0])])
    thefile.close()
    eog_comps = eog_comps1[0]

>Then, similarly, ICs are correlated with FP2. A new txt file is created displaying them. In python it is called "thefile2" and in your data folder it will be called "eog_comps_basedonFp_2.txt".

In [ ]:
if Fp2_bad == 0:
    eog_comps2 = ica.find_bads_eog(raw, ch_name='Fp2')
    thefile2= open(join(fullpath, 'eog_comps_basedonFp_2.txt'),'w')
    thefile2.writelines(["%s\n" % item for item in list(eog_comps2[0])])
    thefile2.close()
    eog_comps += eog_comps2[0]

>Lastly, the ICs are correlated with the eye electrodes (EOG). In the EEG setup we know these as 64 and 65 and in the raw data you might find them under EOG channels. As before, a new txt file is created displaying them. In python it is called "thefile3" and in your data folder it will be called "eog_comps_basedonREALEOG.txt".

In [ ]:
if eog_bad == 0:
    eog_comps3 = ica.find_bads_eog(raw)
    thefile3= open(join(fullpath, 'eog_comps_basedonREALEOG.txt'),'w')
    thefile3.writelines(["%s\n" % item for item in list(eog_comps3[0])])
    thefile3.close()
    eog_comps += eog_comps3[0]

>If you want to, you can combine all 3 outcomes into one file "thefile5" which will be called "eog_comps.txt" in your data folder.
<br><br>
Note: You can simply do this by hand. In this case create a .txt file in the data folder of your participant and simply type in the numbers of the components, ONE per line and with an additional line at the end (aka press enter once more after typing the last component. You don't need to add any brackets or anything, just the numbers. Also remember that python starts counting at 0 (so the first component is component 0).

In [ ]:
thefile5= open(join(fullpath, 'eog_comps.txt'),'w')
thefile5.writelines(["%s\n" % item for item in list(eog_comps)])
thefile5.close()

>The very last command will simply print the final EOG components into the command line for you to see and check.

In [ ]:
print(eog_comps)

# Final remarks

[<a href="#Step-by-step-guide">back to top</a>]


Except for Step 10, the output was created using the following code: preprocessing_selectedblocks_1-30Hz and the dataset of participant M19 in the EVC condition. For the ICA output, the dataset of M08 in the EVC was used in another version of the code ("preprocessing_selectedblocks_0.1-8Hz") due to problems with completing the preprocessing with M19.

The information for this instruction manual was collected from data processing meetings with Katerina herself, her own instructions and various online sources (mentioned below), which can be recommended for getting deeper insights into parts of this preprocessing.


# Sources & additional materials

[<a href="#Step-by-step-guide">back to top</a>]

Cohen, M. X. (2014). Analyzing neural time series data: theory and practice. MIT press.

Information on the python libraries:

Numpy: https://numpy.org/doc/stable/ <br> Pandas: https://pandas.pydata.org/pandas-docs/stable/ <br> OS: https://docs.python.org/3/library/os.html  <br> AST: https://docs.python.org/3/library/ast.html  <br> CSV: https://docs.python.org/3/library/csv.html <br> MNE: https://mne.tools/stable/overview/index.html <br> Pickle: https://www.datacamp.com/community/tutorials/pickle-python-tutorial <br> Logging: https://docs.python.org/3/library/logging.html


Handling bad channels (marking & interpolation): https://mne.tools/dev/auto_tutorials/preprocessing/15_handling_bad_channels.html

Specifications for MNE function:
https://mne.tools/stable/generated/mne.preprocessing.ICA.html

Artefact Correction with ICA: https://mne.tools/0.17/auto_tutorials/plot_artifacts_correction_ica.html
https://mne.tools/stable/generated/mne.preprocessing.ICA.html

How to label artefacts: https://labeling.ucsd.edu/tutorial/labels